In [ ]:
# -*- coding: utf-8 -*-
from __future__ import print_function
import os
import sys
try:
    import cPickle as pickle
except:
    import pickle
import time
import numpy as np
import h5py
import math

from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

from deepst.models.STResNet import stresnet
from deepst.config import Config
import deepst.metrics as metrics

import deepst.datasets.TaxiBJ as taxi
import deepst.datasets.BikeNYC as bike

np.random.seed(1337)  # for reproducibility

from keras import backend as K
import tensorflow as tf

In [ ]:
K.set_image_data_format('channels_first')

In [ ]:
"""
HYPER-PARAMETER FOR EXPERIMENTS' CONFIGURATIONS
"""

### ASSIGN THE DATASET HERE
DATASET = 'taxi'

"""
Assertion to ensure dataset correctness
"""
data_options = ['taxi', 'bike']
assert DATASET in data_options, "Dataset not found"

In [ ]:
"""
Assigning global variables
"""
meta_data=True
### Dataset configuration (Fixed by the authors)
if DATASET == "taxi":
    LOAD_DATA = taxi.load_data
    map_height, map_width = 32, 32  # grid size
    T = 48 # 30 mins
    meteorol_data=True
    holiday_data=True
    lr = 0.0002  # learning rate
    len_closeness = 3  # length of closeness dependent sequence
    len_period = 1  # length of peroid dependent sequence
    len_trend = 1  # length of trend dependent sequence
    days_test = 7 * 4 # Last 4 weeks
    m_factor = 1.0
    _patience = 2
    EPHOCS = 500  # number of epoch at training stage
    EPHOCS_CONT = 100  # number of epoch at training (cont) stage
elif DATASET == 'bike':
    LOAD_DATA = bike.load_data
    map_height, map_width = 16, 8  # grid size
    T = 24 # 1 hour
    meteorol_data=False
    holiday_data=False
    lr = 0.0002  # learning rate
    len_closeness = 3  # length of closeness dependent sequence
    len_period = 4  # length of peroid dependent sequence
    len_trend = 4  # length of trend dependent sequence
    days_test = 10 # Last 10 days
    # For NYC Bike data, there are 81 available grid-based areas, each of
    # which includes at least ONE bike station. Therefore, we modify the final
    # RMSE by multiplying the following factor (i.e., factor).
    nb_area = 81
    m_factor = math.sqrt(1. * map_height * map_width / nb_area)
    _patience = 5
    EPHOCS = 500  # number of epoch at training stage
    EPHOCS_CONT = 100  # number of epoch at training (cont) stage
    
print(m_factor)

In [ ]:
# FIXED PARAMETERS
DATAPATH = Config().DATAPATH  # data path, you may set your own data path with the global envirmental variable DATAPATH
CACHEDATA = True  # cache data or NOT
path_cache = os.path.join(DATAPATH, 'CACHE')  # cache path
batch_size = 32  # batch size

nb_flow = 2  # there are two types of flows: inflow and outflow
# divide data into two subsets: Train & Test
len_test = T * days_test
path_result = 'RET'
path_model = 'MODEL'

In [ ]:
"""
ENSURE DIRECTORIES EXIST
"""
if os.path.isdir(path_result) is False:
    os.mkdir(path_result)
if os.path.isdir(path_model) is False:
    os.mkdir(path_model)
if CACHEDATA and os.path.isdir(path_cache) is False:
    os.mkdir(path_cache)

In [ ]:
"""
Memory management for Keras (tensorflow) so it does not occupy the whole GPU memory
Reference: 
- https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth                               (For Tensorflow 2.0)
- https://kobkrit.com/using-allow-growth-memory-option-in-tensorflow-and-keras-dc8c8081bc96     (For Tensorflow 1.0)
"""
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2" # Using the GPU with ID = 0
# ### For debugging purpose
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU') # Restrict TensorFlow to only use the first GPU
        tf.config.experimental.set_memory_growth(gpus[0], True)    # Dynamic memory allocation to save memory space
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

In [ ]:
def build_model(external_dim, nb_residual_unit=2, bn=False, fusion=True):
    c_conf = (len_closeness, nb_flow, map_height,
              map_width) if len_closeness > 0 else None
    p_conf = (len_period, nb_flow, map_height,
              map_width) if len_period > 0 else None
    t_conf = (len_trend, nb_flow, map_height,
              map_width) if len_trend > 0 else None

    model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf,
                     external_dim=external_dim, nb_residual_unit=nb_residual_unit, bn=bn, fusion=fusion)
    adam = Adam(lr=lr)
    model.compile(loss='mse', optimizer=adam, metrics=[metrics.rmse])
    model.summary()
    # from keras.utils.visualize_util import plot
    # plot(model, to_file='model.png', show_shapes=True)
    return model


def read_cache(fname, pre='preprocessing.pkl'):
    mmn = pickle.load(open(pre, 'rb'))

    f = h5py.File(fname, 'r')
    num = int(f['num'][()])
    X_train, Y_train, X_test, Y_test = [], [], [], []
    for i in range(num):
        X_train.append(f['X_train_%i' % i][()])
        X_test.append(f['X_test_%i' % i][()])
    Y_train = f['Y_train'][()]
    Y_test = f['Y_test'][()]
    external_dim = int(f['external_dim'][()])
    timestamp_train = f['T_train'][()]
    timestamp_test = f['T_test'][()]
    f.close()

    return X_train, Y_train, X_test, Y_test, mmn, external_dim, timestamp_train, timestamp_test


def cache(fname, X_train, Y_train, X_test, Y_test, external_dim, timestamp_train, timestamp_test):
    h5 = h5py.File(fname, 'w')
    h5.create_dataset('num', data=len(X_train))

    for i, data in enumerate(X_train):
        h5.create_dataset('X_train_%i' % i, data=data)
    # for i, data in enumerate(Y_train):
    for i, data in enumerate(X_test):
        h5.create_dataset('X_test_%i' % i, data=data)
    h5.create_dataset('Y_train', data=Y_train)
    h5.create_dataset('Y_test', data=Y_test)
    external_dim = -1 if external_dim is None else int(external_dim)
    h5.create_dataset('external_dim', data=external_dim)
    h5.create_dataset('T_train', data=timestamp_train)
    h5.create_dataset('T_test', data=timestamp_test)
    h5.close()

def main(nb_residual_unit=2, bn=False, fusion=True, epochs=EPHOCS, nb_epoch_cont=EPHOCS_CONT):
    # load data
    print("loading data...")
    ts = time.time()
    fname = os.path.join(DATAPATH, 'CACHE', '{}_C{}_P{}_T{}.h5'.format(
        DATASET, len_closeness, len_period, len_trend))
    pre = 'preprocessing_%s.pkl' % DATASET
    if os.path.exists(fname) and CACHEDATA:
        X_train, Y_train, X_test, Y_test, mmn, external_dim, timestamp_train, timestamp_test = read_cache(
            fname, pre)
        print("load %s successfully" % fname)
    else:
        X_train, Y_train, X_test, Y_test, mmn, external_dim, timestamp_train, timestamp_test = LOAD_DATA(
            T=T, nb_flow=nb_flow, len_closeness=len_closeness, len_period=len_period, len_trend=len_trend, len_test=len_test,
            preprocess_name=pre, 
            meta_data=meta_data, meteorol_data=meteorol_data, holiday_data=holiday_data)
        if CACHEDATA:
            cache(fname, X_train, Y_train, X_test, Y_test,
                  external_dim, timestamp_train, timestamp_test)

    print("\n days (test): ", [v[:8] for v in timestamp_test[0::T]])
    print("\nelapsed time (loading data): %.3f seconds\n" % (time.time() - ts))

    print('=' * 10)
    print("compiling model...")

    ts = time.time()
    model = build_model(external_dim, nb_residual_unit=nb_residual_unit, bn=bn, fusion=fusion)
    hyperparams_name = '{}_c{}.p{}.t{}.resunit{}.lr{}'.format(
        DATASET, len_closeness, len_period, len_trend, nb_residual_unit, lr)
    fname_param = os.path.join('MODEL', '{}.best.h5'.format(hyperparams_name))

    early_stopping = EarlyStopping(monitor='val_rmse', patience=_patience, mode='min')
    model_checkpoint = ModelCheckpoint(
        fname_param, monitor='val_rmse', verbose=0, save_best_only=True, mode='min')

    print("\nelapsed time (compiling model): %.3f seconds\n" %
          (time.time() - ts))

    print('=' * 10)
    print("training model...")
    ts = time.time()
    history = model.fit(X_train, Y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_split=0.1,
                        callbacks=[early_stopping, model_checkpoint],
                        verbose=1)
    model.save_weights(os.path.join(
        'MODEL', '{}.h5'.format(hyperparams_name)), overwrite=True)
    pickle.dump((history.history), open(os.path.join(
        path_result, '{}.history.pkl'.format(hyperparams_name)), 'wb'))
    print("\nelapsed time (training): %.3f seconds\n" % (time.time() - ts))

    print('=' * 10)
    print('evaluating using the model that has the best loss on the valid set')
    ts = time.time()
    model.load_weights(fname_param)
    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[
                           0] // 48, verbose=0)
    print('Train score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
          (score[0], score[1], score[1] * (mmn._max - mmn._min) / 2. * m_factor))
    score = model.evaluate(
        X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
    print('Test score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
          (score[0], score[1], score[1] * (mmn._max - mmn._min) / 2. * m_factor))
    print("\nelapsed time (eval): %.3f seconds\n" % (time.time() - ts))

    print('=' * 10)
    print("training model (cont)...")
    ts = time.time()
    fname_param = os.path.join(
        'MODEL', '{}.cont.best.h5'.format(hyperparams_name))
    model_checkpoint = ModelCheckpoint(
        fname_param, monitor='rmse', verbose=0, save_best_only=True, mode='min')
    history = model.fit(X_train, Y_train, epochs=nb_epoch_cont, verbose=1, batch_size=batch_size, callbacks=[
                        model_checkpoint])
    pickle.dump((history.history), open(os.path.join(
        path_result, '{}.cont.history.pkl'.format(hyperparams_name)), 'wb'))
    model.save_weights(os.path.join(
        'MODEL', '{}_cont.h5'.format(hyperparams_name)), overwrite=True)
    print("\nelapsed time (training cont): %.3f seconds\n" % (time.time() - ts))

    print('=' * 10)
    print('evaluating using the final model')
    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[
                           0] // 48, verbose=0)
    print('Train score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
          (score[0], score[1], score[1] * (mmn._max - mmn._min) / 2. * m_factor))
    ts = time.time()
    score = model.evaluate(
        X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
    print('Test score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
          (score[0], score[1], score[1] * (mmn._max - mmn._min) / 2. * m_factor))
    print("\nelapsed time (eval cont): %.3f seconds\n" % (time.time() - ts))
    
    y_pred = model.predict(X_test)
    return y_pred, Y_test, mmn, m_factor

In [ ]:
def RMSE(y_true, y_pred):
    assert len(y_true) == len(y_pred), "Length of y_true must be equal to y_pred"
    rmse = 0.0
    for i in range(len(y_true)):
        x = (np.sqrt(np.mean((y_true-y_pred)**2)))
        rmse += x
    rmse /= len(y_true)
    return rmse

# Running the Evaluations

In [ ]:
%%notify
%%timeit -n 1 -r 1

y_pred, Y_test, mmn, m_factor = main(nb_residual_unit=2, bn=False)
K.clear_session()
rmse = RMSE(Y_test, y_pred)
print(rmse)
RMSE(Y_test, y_pred) * (mmn._max - mmn._min) / 2. * m_factor

In [ ]:
%%notify
%%timeit -n 1 -r 1
### Model L2
main(nb_residual_unit=2, bn=False)
K.clear_session()

In [ ]:
%%notify
%%timeit -n 1 -r 1
### Model L4
main(nb_residual_unit=4, bn=False)
K.clear_session()

In [ ]:
%%notify
%%timeit -n 1 -r 1
### Model L12
main(nb_residual_unit=12, bn=False)
K.clear_session()

In [ ]:
%%notify
%%timeit -n 1 -r 1
### Model L12-BN
main(nb_residual_unit=12, bn=True)
K.clear_session()

In [ ]:
%%notify
%%timeit -n 1 -r 1
### Model L2-BN
main(nb_residual_unit=2, bn=True)
K.clear_session()

In [ ]:
%%notify
%%timeit -n 1 -r 1
### Model L4-BN
main(nb_residual_unit=4, bn=True)
K.clear_session()